Importar Librerías

In [15]:
import pandas as pd
import numpy as np
import h5py
import os
import torch
#import helper
import matplotlib.pyplot as plt
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch import nn
from collections import OrderedDict

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Definir variables globales

In [16]:
ancho = 160
alto = 129
canales = 3
pixeles = ancho*alto*canales
cantidadClases = 5
clases = ['Banano', 'Granada', 'Kiwi', 'Mango', 'Pitaya']

Definir Función Auxiliar

In [17]:
def view_classify(img, ps):
  ps = ps.data.numpy().squeeze()
  fig, (ax1, ax2) = plt.subplots(figsize=(16,13), ncols=2)
  ax1.imshow(img.numpy().transpose((1, 2, 0)))
  ax1.axis('off')
  ax2.barh(np.arange(cantidadClases), ps)
  ax2.set_aspect(0.1)
  ax2.set_yticks(np.arange(cantidadClases))
  ax2.set_yticklabels(clases, size='medium');
  ax2.set_title('Probabilidad de las Clases')
  ax2.set_xlim(0, 1.1)

  plt.tight_layout()

Importar dataset

In [18]:
#Se transforma a la mitad del tamaño por restricciones de memoria
transform = transforms.Compose([transforms.Resize((ancho, 129)),transforms.ToTensor(),])
current_dir = os.getcwd()
dataset_path = os.path.join(current_dir, 'dataset', 'Frutas', 'Train')
trainset = ImageFolder(root=dataset_path, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

La función de activación será relu para las capas ocultas y softmax para la capa final: https://www.mdpi.com/2079-9292/12/14/3132

Definir la estructura de la red y el forward

In [19]:
class Network(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(pixeles, 20640)
    self.fc2 = nn.Linear(20640, 6880)
    self.fc3 = nn.Linear(6880, 2293)
    self.fc4 = nn.Linear(2293, 764)
    self.fc5 = nn.Linear(764, 254)
    self.fc6 = nn.Linear(254, 84)
    self.fc7 = nn.Linear(84, cantidadClases)
      
  def forward(self, x):
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)
    x = F.relu(x)
    x = self.fc5(x)
    x = F.relu(x)
    x = self.fc6(x)
    x = F.relu(x)
    x = self.fc7(x)
    x = F.softmax(x, dim=1)
    
    return x

model = Network()

Inicializar pesos y bias

In [20]:
model.fc1.bias.data.fill_(0)
model.fc1.weight.data.normal_(std=0.01)

tensor([[ 0.0071,  0.0002,  0.0025,  ..., -0.0082, -0.0102, -0.0102],
        [-0.0190, -0.0124,  0.0025,  ...,  0.0053, -0.0036, -0.0009],
        [-0.0080, -0.0055, -0.0137,  ...,  0.0114,  0.0089, -0.0130],
        ...,
        [-0.0221, -0.0023,  0.0063,  ...,  0.0167,  0.0048, -0.0103],
        [-0.0092,  0.0074, -0.0132,  ...,  0.0002, -0.0030, -0.0044],
        [-0.0037,  0.0060, -0.0181,  ..., -0.0053, -0.0180, -0.0143]])

Construir modelo con capas secuenciales

In [21]:
input_size = pixeles
hidden_sizes = [20640, 6880, 2293, 764, 254, 84]
output_size = cantidadClases

model = nn.Sequential(OrderedDict([
  ('capa1',nn.Linear(input_size, hidden_sizes[0])),
  ('actv1',nn.ReLU()),
  ('capa2',nn.Linear(hidden_sizes[0], hidden_sizes[1])),
  ('actv2',nn.ReLU()),
  ('capa3',nn.Linear(hidden_sizes[1], hidden_sizes[2])),
  ('actv3',nn.ReLU()),
  ('capa4',nn.Linear(hidden_sizes[2], hidden_sizes[3])),
  ('actv4',nn.ReLU()),
  ('capa5',nn.Linear(hidden_sizes[3], hidden_sizes[4])),
  ('actv5',nn.ReLU()),
  ('capa6',nn.Linear(hidden_sizes[4], hidden_sizes[5])),
  ('actv6',nn.ReLU()),
  ('capa7',nn.Linear(hidden_sizes[5], output_size)),
  ('actv7',nn.Softmax(dim=1))
]))

Sequential(
  (capa1): Linear(in_features=61920, out_features=20640, bias=True)
  (actv1): ReLU()
  (capa2): Linear(in_features=20640, out_features=6880, bias=True)
  (actv2): ReLU()
  (capa3): Linear(in_features=6880, out_features=2293, bias=True)
  (actv3): ReLU()
  (capa4): Linear(in_features=2293, out_features=764, bias=True)
  (actv4): ReLU()
  (capa5): Linear(in_features=764, out_features=254, bias=True)
  (actv5): ReLU()
  (capa6): Linear(in_features=254, out_features=84, bias=True)
  (actv6): ReLU()
  (capa7): Linear(in_features=84, out_features=5, bias=True)
  (actv7): Softmax(dim=1)
)


Realizar forward

In [23]:
images,labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)
ps = model.forward(images[0,:].unsqueeze(0))

Calcular las pérdidas, nuestra predicción de error

In [26]:
criterion = nn.NLLLoss()

images,labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)
logps = model(images)
loss = criterion(logps, labels)

print(loss)

tensor(-0.2007, grad_fn=<NllLossBackward0>)
